In [1]:
import os

In [2]:
list_file = list(os.walk('./part'))[0][2]
if '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
if '0.null.txt' in list_file:
    list_file.remove('0.null.txt')
    
list_file.sort()
    
len(list_file)

393

In [3]:
import pandas as pd

In [4]:
df = pd.DataFrame()

df

""


In [5]:
from tqdm import tqdm
from bs4 import BeautifulSoup
from lxml import etree
import json

In [6]:
for file in tqdm(list_file, desc='Progress', ncols=77):
    with open('./part/' + file, 'r', encoding='UTF-8') as f:
        html = f.read()
        
        # = = = = = = = = = = = = = = =
        
        soup = BeautifulSoup(html, 'lxml')
        html = etree.HTML(str(soup))
        
        # = = = = = = = = = = = = = = =

        type_ = html.xpath('//span[@id="lblPartTypeName"]/text()')[0].strip()

        # = = = = = = = = = = = = = = =
        
        part_number = html.xpath('//span[@id="lblProductName"]/text()')[0].strip()
        
        # = = = = = = = = = = = = = = =
        
        description = html.xpath('//span[@id="lblProductDesc"]/text()')[0].strip()
        
        # = = = = = = = = = = = = = = =
        
        application_summary = ''.join(html.xpath('//div[@id="divAppSummary"]/p/text()')).strip().replace(', ', '\n')

        # = = = = = = = = = = = = = = =
        
        src = html.xpath('//a[@id="ProductPic"]/@href')[0].strip()
        
        # = = = = = = = = = = = = = = =
        
        vehicle_url = html.xpath('//input[@id="hidfDetailApp"]/@value')
        if len(vehicle_url) == 0:
            vehicle_url = ''
        else:
            vehicle_url = 'https://www.dormanproducts.com/' + vehicle_url[0].strip()
            
        # = = = = = = = = = = = = = = =
        
        list_oe = [oe.strip() for oe in html.xpath('//section[@id="productOE"]/descendant::tbody/tr/th/text()')]
        list_oe = list(set(list_oe))
        oe = ';'.join(list_oe)

        # = = = = = = = = = = = = = = =
        
        list_tr = html.xpath('//section[@id="productSpec"]/descendant::tr')
        
        # = = = = = = = = = = = = = = =
        
        df_temp = pd.DataFrame([{'序号': int(file.replace('.txt', '')),
                                 'Part_Number': part_number,
                                 'Type': type_,
                                 'Description': description,
                                 'Application_Summary': application_summary,
                                 'Vehicle_1': '',
                                 'Vehicle_2': '',
                                 'OE': oe,
                                 'Pic': '',
                                 'Url': '',
                                 'Vehicle_Url': vehicle_url,
                                 'Src': src}])

        for tr in list_tr:
            name = tr.xpath('./th/text()')[0].strip()[:-1]
            df_temp.loc[0, name[:-1].strip() if name.endswith(':') else name] = tr.xpath('./td/text()')[0].strip()

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

# = = = = = = = = = = = = = = =
        
df = df.drop_duplicates(ignore_index=True)
df = df.sort_values(by=['序号'], ascending=[True]).reset_index(drop=True)
df.to_excel('./part.xlsx', index=False)

df

Progress: 100%|████████████████████████████| 393/393 [00:19<00:00, 20.42it/s]


,序号,Part_Number,Type,Description,Application_Summary,Vehicle_1,Vehicle_2,OE,Pic,Url,...,Vent Line Attached,Vent Quantity,Clamps Included,Fill Pipe Inside Diameter,Vent 1 Inside Diameter,Vent 1 Type,Vent 2 Inside Diameter,Vent 2 Type,Vent 3 Type,Vent 4 Type
0,1,574-091,Fuel Filler Neck,Fuel Filler Neck,Chevrolet 2007-05\nGMC 2007-05,,,10365931,,,...,No,0,,,,,,,,
1,2,574-021,Fuel Filler Neck,Fuel Filler Neck Assembly,Ford Edge 2014-11\nLincoln MKX 2015-11,,,BT4Z9034A,,,...,Yes,1,,,,Rubber,,,,
2,3,574-151,Fuel Filler Neck,Fuel Filler Neck,Acura TL 2008-04\nHonda Accord 2007-03,,,17660SDAA02;17660SDAA01,,,...,No,0,,,,,,,,
3,4,574-103,Fuel Filler Neck,Fuel Filler Neck,Jeep Wrangler 2006-03,,,52100495AH;52100495AE;52100495AK;52100495AF;52...,,,...,Yes,1,No,57.1 mm,6 mm,Steel,8 mm,Steel,,
4,5,574-037,Fuel Filler Neck,Fuel Filler Neck Assembly,Lexus ES300 2003-99\nToyota Camry 2001-97,,,7720133020;7720133050,,,...,Yes,1,,,,Steel,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
388,389,577-370,Fuel Filler Neck,Fuel Filler Neck Tube,Toyota Sienna 2013-11,,,7720108050,,,...,Yes,1,,,,Metal,,,,
389,390,577-323,Fuel Filler Neck,Fuel Filler Neck Tube,Chevrolet Malibu 2012-09,,,20823196,,,...,,,,,,,,,,
390,391,577-322,Fuel Filler Neck,Fuel Tank Filler Neck,Saturn 2005-00,,,22732948;10382365,,,...,,,,,,,,,,
391,392,577-353,Fuel Filler Neck,Fuel Filler Neck Tube,Jeep Grand Cherokee 2013-11,,,52124611AE;52124611AI;52124611AG;52124611AH;52...,,,...,Yes,2,,,,Metal Hose Combination,,Metal Hose Combination,,
